## Global Terrorism Attribution Predictor

In this project I...

Imports and field variables

In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing
import matplotlib.pyplot as plt
data_path = "./data"

Download Global Terrorism Database from Kaggle

In [2]:
api = KaggleApi()
api.authenticate()
api.dataset_download_files("START-UMD/gtd", quiet=False, path=data_path)

gtd.zip: Skipping, found more recently modified local copy (use --force to force download)


Read data into Pandas DataFrame

In [3]:
# skipcols=lambda x: x not in ["extended", "addnotes", "scite1", "scite2", "scite3", "dbsource"]
usecols=["eventid",
         "imonth",
         "iday",
         "country",
         "region",
         "provstate",
         "city",
         "attacktype1",
         "targtype1",
         "targsubtype1",
         "natlty1",
         "gname",
         "weaptype1",
         "ransom"]

data = pd.read_csv(data_path + "/gtd.zip",
                        encoding = "ISO-8859-1",
                        compression='zip',
                        usecols=usecols)

set dTypes

In [4]:
data = pd.DataFrame(data.dropna())
data['attacktype1'] = data.attacktype1.astype('category')
data['targtype1'] = data.targtype1.astype('category')
data['targsubtype1'] = data.targsubtype1.astype('int').astype('category')
data['natlty1'] = data.natlty1.astype('int').astype('category')
data['weaptype1'] = data.weaptype1.astype('category')
data['ransom'] = data.ransom.astype('int').astype('category')

In [5]:
pd.set_option('display.max_columns', None) 
data.head()

,eventid,imonth,iday,country,region,provstate,city,attacktype1,targtype1,targsubtype1,natlty1,gname,weaptype1,ransom
1,197000000002,0,0,130,1,Federal,Mexico city,6,7,45,21,23rd of September Communist League,13,1
2,197001000001,1,0,160,5,Tarlac,Unknown,1,10,54,217,Unknown,13,0
3,197001000002,1,0,78,8,Attica,Athens,3,7,46,217,Unknown,6,0
4,197001000003,1,0,101,4,Fukouka,Fukouka,7,7,46,217,Unknown,8,0
5,197001010002,1,1,217,1,Illinois,Cairo,2,3,22,217,Black Nationalists,5,0


In [6]:
data.describe()

,eventid,imonth,iday,country,region
count,7.297100e+04,72971.000000,72971.000000,72971.000000,72971.000000
mean,1.990256e+11,6.497348,15.402681,142.611873,6.019761
std,1.048609e+09,3.401263,8.807792,129.704451,3.231550
min,1.970000e+11,0.000000,0.000000,4.000000,1.000000
25%,1.983091e+11,4.000000,8.000000,61.000000,3.000000
50%,1.989072e+11,6.000000,15.000000,110.000000,6.000000
75%,1.994122e+11,9.000000,23.000000,183.000000,9.000000
max,2.017123e+11,12.000000,31.000000,1004.000000,12.000000


Filter out unattributed attacks

In [7]:
attributed = data.loc[data['gname'] != "Unknown"]

In [8]:
attributed = pd.DataFrame(attributed.dropna())
attributed = attributed.iloc[:1000]
attributed.head()

,eventid,imonth,iday,country,region,provstate,city,attacktype1,targtype1,targsubtype1,natlty1,gname,weaptype1,ransom
1,197000000002,0,0,130,1,Federal,Mexico city,6,7,45,21,23rd of September Communist League,13,1
5,197001010002,1,1,217,1,Illinois,Cairo,2,3,22,217,Black Nationalists,5,0
6,197001020001,1,2,218,3,Montevideo,Montevideo,1,3,25,218,Tupamaros (Uruguay),5,0
8,197001020003,1,2,217,1,Wisconsin,Madison,7,4,28,217,New Year's Gang,8,0
9,197001030001,1,3,217,1,Wisconsin,Madison,7,2,21,217,New Year's Gang,8,0


In [9]:
# attributed.head()
# to_fit = attributed.select_dtypes(include=['object'])
# le = preprocessing.LabelEncoder()
# attributed.apply(le.fit_transform)
# attributed.head()
# attributed = to_fit.apply(le.fit_transform)

In [10]:
# 1. INSTANTIATE
# enc = preprocessing.OneHotEncoder()

# # 2. FIT
# enc.fit(attributed)

# 3. Transform
# onehotlabels = enc.transform(attributed).toarray()
attributed = pd.get_dummies(attributed)
attributed.shape

(1000, 888)

In [11]:
# plt.imshow(attributed.corr(), cmap=plt.cm.Reds, interpolation='nearest')
# plt.colorbar()
# tick_marks = [i for i in range(len(attributed.columns))]
# plt.xticks(tick_marks, attributed.columns, rotation='vertical')
# plt.yticks(tick_marks, attributed.columns)
# plt.show()

Making a copy of the attributeddataframe which does not contain the dependent variable

In [12]:
independent_vars = attributed.copy()
independent_vars = independent_vars[independent_vars.columns.drop(list(independent_vars.filter(regex='gname')))]
independent_vars.shape

(1000, 780)

Making a dataframe with of the dependent variables (organizations to which the attacks were attribued)

In [13]:
dependent_vars = pd.DataFrame(attributed.loc[:,list(attributed.filter(regex='gname'))])
dependent_vars.shape

(1000, 108)

Split the data into training and test sets

In [14]:
independant_size = independent_vars.shape
dependant_size = dependent_vars.shape
if independant_size[0] != dependant_size[0]:
    raise ValueError('Independent and Dependant DFs do not match')

split_size = int(independant_size[0] * 0.7)

training_independant_vars = independent_vars.iloc[:split_size]
test_independant_vars = independent_vars.iloc[split_size:]
print(training_independant_vars.shape)
print(test_independant_vars.shape)

training_dependant_vars = dependent_vars.iloc[:split_size]
test_dependant_vars = dependent_vars.iloc[split_size:]

print(training_dependant_vars.shape)
print(test_dependant_vars.shape)

(700, 780)
(300, 780)
(700, 108)
(300, 108)


TODO: Implement Training model

In [15]:
params = [{}, {"loss": "log", "penalty": "l2", 'n_iter':1000}]
reg = SGDClassifier()
reg.fit(training_independant_vars, training_dependant_vars)


/Users/steven/anaconda3/envs/terrorism/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


ValueError: bad input shape (700, 108)